#Importing the libraries


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.15.0'

#Data Preprocessing

In [3]:
#importing the dataset

data = pd.read_csv('Churn_Modelling.csv')
X = data.drop(columns = ['RowNumber', 'CustomerId', 'Surname', 'Exited'])
y = data['Exited']

In [4]:
data.isnull().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [5]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


###Encoding the categorical data

In [6]:
#Lable Encoding the 'Gender' column
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
X['Gender'] = labels.fit_transform(X['Gender'])

In [7]:
X.sample(n=10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
8302,850,France,0,39,2,0.00,2,1,0,179451.42
5580,616,Spain,1,30,2,0.00,2,1,0,199099.51
7936,468,France,1,22,2,0.00,2,1,0,28123.99
4874,703,France,1,43,8,0.00,2,1,0,9704.66
7450,712,Spain,0,39,5,163097.55,2,1,1,23702.42
9304,616,France,1,40,9,0.00,2,0,0,93717.55
8344,520,Germany,1,43,3,150805.17,3,0,1,25333.03
4563,705,Germany,0,64,3,153469.26,3,0,0,146573.66
7624,728,France,0,69,1,0.00,2,1,1,131804.86
8701,673,France,1,37,2,0.00,1,1,1,13624.02


In [8]:
#One hot Encoding the 'Geography' column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

###Train-Test-Split into Training and Testing set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

###Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

#Building the ANN


###Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

###Adding the layers

In [12]:
ann.add(tf.keras.layers.Dense(units = 5 , activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 5 , activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))

#Training the ANN

###Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics =['accuracy'])

###Training the ANN with the training data

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.5549 - accuracy: 0.7939
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4865 - accuracy: 0.7977
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4639 - accuracy: 0.7977
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.7977
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.7977
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4362 - accuracy: 0.7980
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4319 - accuracy: 0.7987
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4286 - accuracy: 0.8000
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.8007
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4219 - accura

#Predicting the result



In [22]:
print(ann.predict(scl.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.6)

1/1 [==============================] - 0s 24ms/step
[[False]]


###Predicting the test set result

In [25]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.6)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)),1))

94/94 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 1]]


###Confusion matrix

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2329   50]
 [ 354  267]]


0.8653333333333333